In [1]:
# import library
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
import sys
import os
import httpx

In [ ]:
import concurrent.futures
from datetime import datetime
import glob
import warnings
from rich.progress import track
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'rich'

In [ ]:
def make_api_request(api_endpoint, params):
    with httpx.Client() as client:
        # Make the GET request to the API
        response = client.get(api_endpoint, params=params)
        if response.status_code == 200:
            return response.json()
        print("Error: Failed to retrieve data from API")
        return None

In [ ]:
BASE_URL_FMP = "https://financialmodelingprep.com/api/v3"
BASE_URL_BINANCE = "https://fapi.binance.com/fapi/v1/"

FMP_API_KEY = ""
BINANCE_API_KEY = ""


def get_historical_price_full_crypto(symbol):
    api_endpoint = f"{BASE_URL_FMP}/historical-price-full/crypto/{symbol}"
    params = {"apikey": FMP_API_KEY}
    return make_api_request(api_endpoint, params)


def get_historical_price_full_stock(symbol):
    api_endpoint = f"{BASE_URL_FMP}/historical-price-full/{symbol}"
    params = {"apikey": FMP_API_KEY}

    return make_api_request(api_endpoint, params)

def get_SP500():
    api_endpoint = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    data = pd.read_html(api_endpoint)
    return list(data[0]['Symbol'])

def get_all_crypto():
    """
    All possible crypto symbols
    """
    return [
        "BTCUSD",
        "ETHUSD",
        "LTCUSD",
        "BCHUSD",
        "XRPUSD",
        "EOSUSD",
        "XLMUSD",
        "TRXUSD",
        "ETCUSD",
        "DASHUSD",
        "ZECUSD",
        "XTZUSD",
        "XMRUSD",
        "ADAUSD",
        "NEOUSD",
        "XEMUSD",
        "VETUSD",
        "DOGEUSD",
        "OMGUSD",
        "ZRXUSD",
        "BATUSD"
        "USDTUSD",
        "LINKUSD",
        "BTTUSD",
        "BNBUSD",
        "ONTUSD",
        "QTUMUSD",
        "ALGOUSD",
        "ZILUSD",
        "ICXUSD",
        "KNCUSD",
        "ZENUSD",
        "THETAUSD",
        "IOSTUSD",
        "ATOMUSD",
        "MKRUSD",
        "COMPUSD",
        "YFIUSD",
        "SUSHIUSD",
        "SNXUSD",
        "UMAUSD",
        "BALUSD",
        "AAVEUSD",
        "UNIUSD",
        "RENBTCUSD",
        "RENUSD",
        "CRVUSD",
        "SXPUSD",
        "KSMUSD",
        "OXTUSD",
        "DGBUSD",
        "LRCUSD",
        "WAVESUSD",
        "NMRUSD",
        "STORJUSD",
        "KAVAUSD",
        "RLCUSD",
        "BANDUSD",
        "SCUSD",
        "ENJUSD",
    ]

def get_financial_statements_lists():
    api_endpoint = f"{BASE_URL_FMP}/financial-statement-symbol-lists"
    params = {"apikey": FMP_API_KEY}
    return make_api_request(api_endpoint, params)

In [ ]:
from transformers import pipeline

def summarizer(text: str):
    # Instantiate a pipeline for summarization
    summarization = pipeline("summarization")
    return summarization(text)[0]['summary_text']


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

def calculate_sentiment(text: str):
    tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
    model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

    inputs = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        return_token_type_ids=False,
        return_attention_mask=True,
        truncation=True,
        return_tensors='pt'
    )

    output = model(**inputs)
    logits = output.logits.detach().numpy()

    sentiment = torch.argmax(output.logits)
    score = torch.softmax(output.logits, dim=1).detach().numpy()[0][sentiment]

    if sentiment == 0:
        sentiment = 'negative'
    elif sentiment == 1:
        sentiment = 'neutral'
    else:
        sentiment = 'positive'

    # If the sentiment is neutral and score > 0.8, return bad article
    if sentiment == 'neutral' and score > 0.8:
        return "Bad article, not in context"

    return sentiment, score

In [ ]:
import httpx
from datetime import datetime

async def fetch_rss_feed(api_key, start_date, end_date, limit=10):
    url = "https://financialmodelingprep.com/api/v4/rss_feed_8k"

    params = {
        "from": start_date.strftime('%Y-%m-%d'),
        "to": end_date.strftime('%Y-%m-%d'),
        "limit": limit,
        "apikey": api_key
    }

    async with httpx.AsyncClient() as client:
        response = await client.get(url, params=params)
    return response.json()

async def fetch_articles(api_key: str, query: str, lang: str, country: str, max_results: int):
    url = f'https://gnews.io/api/v4/search?q={query}&lang={lang}&country={country}&max={max_results}&apikey={api_key}'
    async with httpx.AsyncClient() as client:
        response = await client.get(url)
    data = response.json()
    return data['articles']

async def print_articles(api_key: str, query: str, lang: str, country: str, max_results: int):
    articles = await fetch_articles(api_key, query, lang, country, max_results)
    for article in articles:
        print("Title: ", article['title'])
        print("Description: ", article['description'])


In [ ]:
def execute_trading_strategy(
    companies_to_follow: list, 
    fetch_news: callable, 
    analyze_sentiment: callable, 
    buy_stock: callable, 
    sell_stock: callable, 
    sentiment_threshold: float, 
    buy_sell_percentage: float
):
    for company in companies_to_follow:
        news_articles = fetch_news(company)

        positive_count = 0
        negative_count = 0

        for article in news_articles:
            sentiment_score = analyze_sentiment(article)
            
            if sentiment_score > sentiment_threshold:
                positive_count += 1
            elif sentiment_score < -sentiment_threshold:
                negative_count += 1

        total_count = len(news_articles)

        if total_count > 0:
            positive_ratio = positive_count / total_count
            negative_ratio = negative_count / total_count

            if positive_ratio > buy_sell_percentage:
                buy_stock(company)
            elif negative_ratio > buy_sell_percentage:
                sell_stock(company)
